In [ ]:
# Google Colab + Github Connect
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git config --global user.email "silasskim@gmail.com"
!git config --global user.name "Rafflesia"

In [ ]:
from getpass import getpass
import os
username = "your_github_username"
token = getpass('Enter your GitHub token: ')
repo_name = "your_repository_name"

!git clone https://{token}@github.com/{username}/{repo_name}.git

In [ ]:
%cd /content/{repo_name}

In [ ]:
# The main code Starts
!pip install selenium
!pip install webdriver_manager
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

In [ ]:
# Define Process
def extract_market_data(target_date='2025-01-29', target_contract='FTS PPA 26/35'):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    driver = webdriver.Chrome(options=chrome_options)

    try:
        url = f"https://www.omip.pt/en/dados-mercado?date={target_date}&product=EL&zone=ES&instrument=FTS&maturity=PPA"
        driver.get(url)
        print(f"Page loaded successfully for date: {target_date}")
        time.sleep(5)

        rows = driver.find_elements(By.CSS_SELECTOR, "tr.odd, tr.even")
        all_data = []

        print(f"\nLooking for contract: {target_contract}")

        for row in rows:
            try:
                contract_element = row.find_element(By.TAG_NAME, "strong")
                if contract_element:
                    contract_name = contract_element.text
                    if contract_name == target_contract:
                        print(f"Processing contract: {contract_name}")

                        chart_button = row.find_element(By.CSS_SELECTOR, "a.chart-down")
                        chart_url = chart_button.get_attribute("href")

                        full_history_url = chart_url.replace('/0/0', '/0/1')
                        print(f"Full history URL: {full_history_url}")

                        # Full History
                        driver.execute_script(f"window.open('{full_history_url}', '_blank');")
                        time.sleep(3)
                        driver.switch_to.window(driver.window_handles[-1])

                        chart_div = driver.find_element(By.CLASS_NAME, "charts-highchart")
                        chart_data = chart_div.get_attribute("data-chart")

                        import json
                        data = json.loads(chart_data)

                        # Extract
                        price_series = next(series for series in data['series'] if series['name'] == 'Price (€)')

                        # DataFrame
                        for point in price_series['data']:
                            timestamp = point[0] / 1000
                            price = point[1]
                            date = pd.to_datetime(timestamp, unit='s')

                            all_data.append({
                                'Date': date,
                                'Price': price
                            })

                        # Close tab and switch back
                        driver.close()
                        driver.switch_to.window(driver.window_handles[0])
                        break

            except Exception as e:
                print(f"Error processing row: {e}")
                continue

        # Create DataFrame
        df = pd.DataFrame(all_data)
        df = df.sort_values('Date')

        print(f"\nDate range for {target_contract}:")
        print("Earliest date:", df['Date'].min())
        print("Latest date:", df['Date'].max())

        return df

    finally:
        driver.quit()

# Extract wanted Date / Type
target_date = '2025-01-29'
target_contract = 'FTS PPA 26/35'
df = extract_market_data(target_date, target_contract)

print("\nExtracted Data:")
print(df)

# Export CSV
filename = f'ppahistory{target_contract}{target_date}.csv'.replace('/', '')
df.to_csv(filename, index=False)
from google.colab import files
files.download(filename)

In [ ]:
!pwd
!ls
!git status

In [ ]:
!find /content -name "Selenium_PPA_Code.ipynb"

In [ ]:
!cp /content/Selenium_PPA_Code.ipynb /content/Py_Selenm_PPA/

In [ ]:
!cp "/content/drive/MyDrive/Colab Notebooks/Selenium_PPA_Code.ipynb" /content/Py_Selenm_PPA/

In [ ]:
!git add Selenium_PPA_Code.ipynb
!git commit -m "Add Selenium automation notebook"
!git push

In [ ]:
!git add Selenium_PPA_Code.ipynb
!git commit -m "Clear all Outputs"
!git push